In [ ]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

from dataset.graph_builder import GraphBuilder

builder = GraphBuilder(dataset_paths=["data/BALANCED/test.json"], dataset_names=["BALANCED"], dataset_type='test')
builder()

In [ ]:
builder.display_graph(1)

In [ ]:
builder.save_graphs('data/BALANCED/graphs_augmented')

In [ ]:
import os 

os.path.isdir('data/UNIFIED')

In [ ]:
import torch 

x = torch.load(os.path.join('data/UNIFIED/dev', '0.pt'))
x 

In [ ]:
from utils import display_graph 

display_graph(x)